# LLM.int8() 이해 코드

Author : 정상근 (hugmanskj@gmail.com)

> 알림>
> 교육용 목적으로 제작된 코드입니다.


이 노트북LLM.int() 라이브러리 즉 bitsandbytes가 내부적으로 어떻게 작동되는지를 간단한 코드를 통해 알아봅니다.

이 코드는 [Huggingface 튜로리얼]("https://huggingface.co/blog/hf-bitsandbytes-integration")을 참고했음을 밝age



In [1]:
!pip install -q bitsandbytes==0.42.0


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [2]:
import torch
import torch.nn as nn

import bitsandbytes
from bitsandbytes.nn import Linear8bitLt

# FP16 Model 구현

아주 단순한 형태의 fp16 모델을 구현합니다.
16이라고 했지만, 사실은 32입니다. int8과의 설명을 위해서입니다.

In [3]:
input_32 = torch.randn((1, 64), dtype=torch.float32)

이 값은 향후 두개의 서로 다른 모델에 각각 수행해 볼 입력값입니다.

In [4]:
fp16_model = nn.Sequential(
    nn.Linear(64, 64),
    nn.Linear(64, 64)
)

여기 단계에서 훈련을  수행하거나 해서, parameter를 업데이트 했다고 가정합니다.

In [5]:
fp16_model

Sequential(
  (0): Linear(in_features=64, out_features=64, bias=True)
  (1): Linear(in_features=64, out_features=64, bias=True)
)

In [6]:
fp16_model[0].weight

Parameter containing:
tensor([[-0.0561,  0.0689,  0.0916,  ..., -0.0364,  0.0712, -0.1168],
        [-0.0248,  0.0605, -0.1139,  ...,  0.0806,  0.0691, -0.0178],
        [ 0.1028, -0.0730,  0.0527,  ..., -0.0331,  0.0803, -0.0604],
        ...,
        [ 0.0710, -0.0079,  0.1024,  ..., -0.0246, -0.0338,  0.0118],
        [ 0.0906,  0.0995, -0.0781,  ...,  0.0918,  0.0336, -0.0694],
        [-0.1079,  0.0827,  0.0991,  ...,  0.0661,  0.0585, -0.0247]],
       requires_grad=True)

In [7]:
fp16_model[0].weight.dtype

torch.float32

In [8]:
fp16_hidden_states = fp16_model(input_32)

In [9]:
fp16_hidden_states

tensor([[-0.6126, -0.0802,  0.2433,  0.1738,  0.2713,  0.3490,  0.1624, -0.0417,
         -0.2375, -0.2051, -0.6106, -0.4932,  0.1889, -0.2735, -0.1773,  0.0590,
          0.0077, -0.0255, -0.3776, -0.0664, -0.1487, -0.2725,  0.4849, -0.4641,
          0.0473,  0.5667, -0.0265,  0.4313,  0.3749, -0.1647, -0.4747, -0.2956,
          0.0737,  0.3275, -0.0136, -0.2302,  0.2805, -0.1753,  0.2930, -0.2223,
         -0.3790,  0.2329, -0.4455, -0.2961,  0.7046, -0.4024, -0.1282,  0.1875,
         -0.9582,  0.7223,  0.0994, -0.0556,  0.2216, -0.0059, -0.0220, -0.5093,
          0.0613,  0.1409,  0.1279, -0.1777,  0.1569, -0.0951, -0.4302, -0.1716]],
       grad_fn=<AddmmBackward0>)

In [27]:
fp16_model[0].weight

Parameter containing:
tensor([[-0.0561,  0.0689,  0.0916,  ..., -0.0364,  0.0712, -0.1168],
        [-0.0248,  0.0605, -0.1139,  ...,  0.0806,  0.0691, -0.0178],
        [ 0.1028, -0.0730,  0.0527,  ..., -0.0331,  0.0803, -0.0604],
        ...,
        [ 0.0710, -0.0079,  0.1024,  ..., -0.0246, -0.0338,  0.0118],
        [ 0.0906,  0.0995, -0.0781,  ...,  0.0918,  0.0336, -0.0694],
        [-0.1079,  0.0827,  0.0991,  ...,  0.0661,  0.0585, -0.0247]],
       requires_grad=True)

## 모델 저장

In [10]:
torch.save(fp16_model.state_dict(), "model.pt")

# Int8 Model 구현

이제 8 bit 모델을 구현해봅시다.

In [11]:
int8_model = nn.Sequential(
    Linear8bitLt(64, 64, has_fp16_weights=False),
    Linear8bitLt(64, 64, has_fp16_weights=False)
)

- has_fp16_weights 는 기본적으로 True가 default 값입니다.
- True로 셋팅되어 있으면 Int8/FP16 mixed precision 으로 작동됩니다.

여기서는 int8 로 작동되는 것을 테스트 하기 위해 강제로 False로 셋팅하였습니다.

## 16 bit 모델을 8 bit 모델로 로딩
이 단계에서는 단순 모델 로딩만 되지 어떠한 일도 벌어지지 않습니다.

In [12]:
int8_model.load_state_dict(torch.load("model.pt"))

<All keys matched successfully>

In [13]:
int8_model

Sequential(
  (0): Linear8bitLt(in_features=64, out_features=64, bias=True)
  (1): Linear8bitLt(in_features=64, out_features=64, bias=True)
)

In [14]:
before_values = int8_model[0].weight.detach().clone()
before_values

tensor([[-0.0561,  0.0689,  0.0916,  ..., -0.0364,  0.0712, -0.1168],
        [-0.0248,  0.0605, -0.1139,  ...,  0.0806,  0.0691, -0.0178],
        [ 0.1028, -0.0730,  0.0527,  ..., -0.0331,  0.0803, -0.0604],
        ...,
        [ 0.0710, -0.0079,  0.1024,  ..., -0.0246, -0.0338,  0.0118],
        [ 0.0906,  0.0995, -0.0781,  ...,  0.0918,  0.0336, -0.0694],
        [-0.1079,  0.0827,  0.0991,  ...,  0.0661,  0.0585, -0.0247]])

In [15]:
int8_model = int8_model.to(0) # Quantization happens here

위에서 ~~ model. to ( 0 ) 를 통해 GPU 에 할당하는 순간 Quantization 이 수행됩니다.

In [16]:
after_values = int8_model[0].weight
after_values

Parameter containing:
Parameter(Int8Params([[ -59,   72,   96,  ...,  -38,   75, -123],
            [ -27,   65, -123,  ...,   87,   74,  -19],
            [ 108,  -76,   55,  ...,  -35,   84,  -63],
            ...,
            [  73,   -8,  105,  ...,  -25,  -35,   12],
            [  92,  101,  -80,  ...,   94,   34,  -71],
            [-116,   89,  107,  ...,   71,   63,  -27]], device='cuda:0',
           dtype=torch.int8))

- 정수화 되어 있는것을 알 수 있습니다.
- 또한 [-127, 127] 사이의 값으로 조정되어 있는것을 볼 수 있습니다. 

In [17]:
dequantized_values = (int8_model[0].weight.CB * int8_model[0].weight.SCB) / 127
dequantized_values

tensor([[-0.0562,  0.0669,  0.0917,  ..., -0.0369,  0.0735, -0.1140],
        [-0.0257,  0.0604, -0.1175,  ...,  0.0845,  0.0726, -0.0176],
        [ 0.1029, -0.0706,  0.0525,  ..., -0.0340,  0.0824, -0.0584],
        ...,
        [ 0.0696, -0.0074,  0.1003,  ..., -0.0243, -0.0343,  0.0111],
        [ 0.0877,  0.0938, -0.0764,  ...,  0.0913,  0.0333, -0.0658],
        [-0.1105,  0.0827,  0.1022,  ...,  0.0689,  0.0618, -0.0250]],
       device='cuda:0')

## Quantization 전의 값과 복원된 값의 비교

In [18]:
before_values[0][:10]

tensor([-0.0561,  0.0689,  0.0916, -0.0090, -0.0283,  0.0620,  0.1091, -0.0365,
        -0.0449,  0.0352])

In [19]:
dequantized_values[0][:10]

tensor([-0.0562,  0.0669,  0.0917, -0.0083, -0.0293,  0.0639,  0.1085, -0.0370,
        -0.0457,  0.0357], device='cuda:0')

완전히 같지는 않지만 상당히 유사한 값이 나오는 것을 볼 수 있습니다.

# Inference 수행하기

In [20]:
int8_hidden_states = int8_model(input_32.to(torch.device('cuda',0)))

/home/ubuntu/.local/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [21]:
int8_hidden_states

tensor([[-0.6119, -0.0871,  0.2454,  0.1692,  0.2678,  0.3503,  0.1592, -0.0454,
         -0.2346, -0.2103, -0.6112, -0.4930,  0.1819, -0.2722, -0.1743,  0.0647,
          0.0164, -0.0271, -0.3724, -0.0641, -0.1483, -0.2754,  0.4856, -0.4665,
          0.0507,  0.5644, -0.0308,  0.4312,  0.3680, -0.1601, -0.4701, -0.2946,
          0.0793,  0.3237, -0.0131, -0.2273,  0.2826, -0.1716,  0.2944, -0.2272,
         -0.3775,  0.2295, -0.4467, -0.2918,  0.6992, -0.4031, -0.1279,  0.1923,
         -0.9535,  0.7186,  0.1012, -0.0607,  0.2196, -0.0122, -0.0287, -0.5051,
          0.0547,  0.1408,  0.1348, -0.1783,  0.1618, -0.1030, -0.4306, -0.1704]],
       device='cuda:0', grad_fn=<MatMul8bitLtBackward>)

In [22]:
int8_hidden_states.dtype

torch.float32

# 16bit 값과 8bit값 비교하기

In [23]:
fp16_hidden_states[0,:10]

tensor([-0.6126, -0.0802,  0.2433,  0.1738,  0.2713,  0.3490,  0.1624, -0.0417,
        -0.2375, -0.2051], grad_fn=<SliceBackward0>)

In [24]:
int8_hidden_states[0,:10]

tensor([-0.6119, -0.0871,  0.2454,  0.1692,  0.2678,  0.3503,  0.1592, -0.0454,
        -0.2346, -0.2103], device='cuda:0', grad_fn=<SliceBackward0>)

거의 유사한 값이 feed-forward를 통해 계산됨을 알 수 있습니다.

In [25]:
fp16_hidden_states.argmax()

tensor(49)

In [26]:
int8_hidden_states.argmax()

tensor(49, device='cuda:0')

argmax로 살펴보면, 가장 큰 값을 가지는 위치가 같은 위치임을 역시 알 수 있습니다.